

<h1 align="center">
    NYC Airbnb Market Analysis & Revenue Optimization
</h1>
<h3 align="center">
    Jason Shuler
</h3>


#### Import Libraries


In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import sqlite3

### Connect to the Database


In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect('airbnb.db')
cursor = conn.cursor()

#### Load the Dataset

In [ ]:
data_path = 'AB_NYC_2019.csv'
df = pd.read_csv(data_path)
df.to_sql('listings', conn, if_exists='replace', index=False)

48895

- Found the top 10 hosts with the highest average ratings.

In [ ]:
query = """
SELECT host_name, SUM(number_of_reviews) AS num_reviews, AVG(reviews_per_month) AS avg_rating
FROM listings
GROUP BY host_name
ORDER BY avg_rating DESC
LIMIT 10;
"""

In [ ]:
result = pd.read_sql_query(query, conn)
print("\nAnalyzing Host Performance:")
print(result)


Analyzing Host Performance:
      host_name  num_reviews  avg_rating
0       Row NYC          305   18.620000
1       Nalicia         1046   18.126667
2          Dona         1205   13.990000
3       Aisling           17   13.420000
4        Malini          392   13.150000
5       Anabell          259   12.990000
6       Miss Dy          434   12.840000
7         Meeks           15   11.840000
8  Britt & Greg           16   11.160000
9         Hamad          134   10.860000


- Show the month, average price, and number of listings for each month.

In [ ]:
query = """
SELECT strftime('%m', last_review) AS month, AVG(price) AS avg_price,
COUNT(*) AS num_listings FROM listings
GROUP BY month
ORDER BY avg_price DESC;
"""

In [ ]:
result = pd.read_sql_query(query, conn)
print("Analyzing Monthly Price Trends:")
print(result)

Analyzing Monthly Price Trends:
   month   avg_price  num_listings
0   None  192.919021         10052
1     12  160.796610          1770
2     02  155.716883           770
3     01  150.478904          2773
4     09  149.494463          1535
5     07  142.290382          5937
6     10  142.065330          1546
7     06  140.769667         13589
8     04  139.504979          2109
9     11  138.532815          1158
10    05  137.909594          4701
11    03  136.836957          1288
12    08  132.133173          1667


-  Found the top 5 neighborhoods with the highest price variability.
-  Displayed neighborhood and price variability.


In [ ]:
query = """
WITH price_variability AS (
  SELECT neighbourhood, AVG(price) AS avg_price, MAX(price) AS max_price, MIN(price) AS min_price
  FROM listings
  GROUP BY neighbourhood
)
SELECT neighbourhood, max_price - min_price AS price_variability FROM price_variability
ORDER BY price_variability DESC
LIMIT 5;
"""

In [ ]:
result = pd.read_sql_query(query, conn)
print("Advanced Aggregation:")
print(result)

Advanced Aggregation:
     neighbourhood  price_variability
0       Greenpoint              10000
1  Upper West Side               9990
2          Astoria               9975
3  Lower East Side               9970
4      East Harlem               9969


- Calculated days since the last review
- Displayed listing ID, name, and days since the last review.

In [ ]:
query = """
SELECT id, name, ROUND(julianday('now') - julianday(last_review), 0) AS days_since_last_review
FROM listings
ORDER BY days_since_last_review DESC
LIMIT 10;
"""

In [ ]:
result = pd.read_sql_query(query, conn)
print("\nAdvanced Data Manipulation:")
print(result)


Advanced Data Manipulation:
       id                                               name  \
0   74860  Sunlit and Cozy Williamsburg/Greenpoint, Brooklyn   
1   40039                 Luxurious Condo in DUBMO with View   
2   81739                  Loft w/ Terrace @ Box House Hotel   
3   28396                  Modern Apt with Spectacular Views   
4   32363                 Fully Furnished Basement Apartment   
5   27883                             East Village Sanctuary   
6    7801                   Sweet and Spacious Brooklyn Loft   
7  229874                     Oversized Studio in Park Slope   
8   98330                LOVELY APARTMENT IN THE HEART OF NY   
9  464231                     Large Room w/ Private Entrance   

   days_since_last_review  
0                  5390.0  
1                  5362.0  
2                  5345.0  
3                  5216.0  
4                  5215.0  
5                  5133.0  
6                  5115.0  
7                  5110.0  
8             

- Calculated total revenue generated by each host and ranked them based on revenue.
- Showed the host name, total revenue, and their rank.

In [ ]:
query = """
SELECT host_name, SUM(price * minimum_nights) AS total_revenue,
RANK() OVER (ORDER BY SUM(price * minimum_nights) DESC) AS revenue_rank
FROM listings
GROUP BY host_name
ORDER BY total_revenue DESC
LIMIT 10;
"""

In [ ]:
result = pd.read_sql_query(query, conn)
print("Identifying Top Hosts by Revenue:")
print(result)

Identifying Top Hosts by Revenue:
      host_name  total_revenue  revenue_rank
0    Blueground        2258580             1
1         Jenny        1215111             2
2          Kara        1172637             3
3           Amy        1153867             4
4      Kathrine        1002082             5
5  Sonder (NYC)         950453             6
6         Iveta         857750             7
7        Noelle         732930             8
8       Pranjal         664320             9
9       Michael         610539            10
